In [1]:
from iliad_network.components.analyzers import SentimentAnalyzer
from iliad_network.components.preprocessors import TextPreProcessor
from iliad_network.components.readers import LocalReader

In [2]:
!python --version

Python 3.12.8


In [3]:
reader = LocalReader()
try:
    poem_text = reader.run()
    clean_text = poem_text.content  # Access the clean text
except ValueError as e:
    print(f"Validation Error: {e}")

In [4]:
preprocessor = TextPreProcessor(model_name="en_core_web_sm")
iliad_sentences, iliad_words = preprocessor.clean_text(clean_text)
characters = preprocessor.extract_characters(clean_text)

In [5]:
analyzer = SentimentAnalyzer(preprocessor=preprocessor)

# Example usage
char = "Achilles"  # Example character
achilles_sentiments = analyzer.analyze(iliad_sentences, char)

# Print a few example sentences with sentiment scores
for sentence, sentiment in achilles_sentiments[:5]:
    print(f"Sentence: {sentence} | Sentiment: {sentiment}")

Sentence: ----------------------------------------------------------------------  BOOK I  Sing, O goddess, the anger of Achilles son of Peleus, that brought countless ills upon the Achaeans. | Sentiment: -0.35
Sentence: Many a brave soul did it send hurrying down to Hades, and many a hero did it yield a prey to dogs and vultures, for so were the counsels of Jove fulfilled from the day on which the son of Atreus, king of men, and great Achilles, first fell out with one another.    | Sentiment: 0.4490740740740741
Sentence: For nine whole days he shot his arrows among the people, but upon the tenth day Achilles called them in assembly- moved thereto by Juno, who saw the Achaeans in their death-throes and had compassion upon them. | Sentiment: 0.2
Sentence: With all sincerity and goodwill he addressed them thus:-   "Achilles, loved of heaven, you bid me tell you about the anger of King Apollo, I will therefore do so; but consider first and swear that you will stand by me heartily in word a

In [9]:
sentences = [
    "Alice and Bob are friends.",
    "Alice and Charlie are friends.",
    "Bob and Charlie have a disagreement.",
]

char = "Bob"

cooccurrence_matrix = analyzer.build_cooccurrence_matrix(sentences)

In [12]:
assert not cooccurrence_matrix.empty
assert cooccurrence_matrix.loc["Alice", "Bob"] == 1
assert cooccurrence_matrix.loc["Bob", "Charlie"] == 2
assert cooccurrence_matrix.loc["Alice", "Charlie"] == 1

AssertionError: 